In [34]:
import networkx as nx
import random as rand
import numpy as np
import matplotlib.pyplot as plt
from kernighan_lin import kernighan_lin_bisection
import networkx as nx
from stellargraph import StellarGraph
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from tensorflow.keras import layers, optimizers, losses, Model
import pandas as pd
import os

from stellargraph.layer import MeanPoolingAggregator

In [35]:
# We can get from timeclock
SEED_OF_RANDOM = 2524
rand.seed(SEED_OF_RANDOM)

In [36]:
def create_connected_graph(node_list):
    # Create an empty graph
    G = nx.Graph()

    # Add nodes from the specific node list
    G.add_nodes_from(node_list)

    # Add edges to connect the nodes and ensure connectivity
    while not nx.is_connected(G):
        # Randomly select two nodes from the graph
        node1 = rand.choice(list(G.nodes()))
        node2 = rand.choice(list(G.nodes()))

        # Add an edge between the selected nodes
        G.add_edge(node1, node2)

    return G

In [37]:
# simulate graph creation

def simulate_graph_creation(min_size, max_size, min_edge_between, max_edge_between):
        
    # Creating Graphs
    NUMBER_OF_NODES = rand.randint(min_size, max_size)
        
    nodeList_1 = np.arange(1, (NUMBER_OF_NODES+1)/2)
    nodeList_2 = np.arange((NUMBER_OF_NODES+1)/2, NUMBER_OF_NODES+1)
        
    connected_graph_1 = create_connected_graph(nodeList_1)
    connected_graph_2 = create_connected_graph(nodeList_2)
        
    RENAME_1 = 'G_1_' 
    RENAME_2 = 'G_2_'
    G_union = nx.union(connected_graph_1, connected_graph_2, rename = (RENAME_1, RENAME_2))
        
    # Setting Edges
    EDGES_BETWEEN_PARTITIONS = rand.randint(min_edge_between, max_edge_between)
        
    for i in range (EDGES_BETWEEN_PARTITIONS):
        node1 = RENAME_1 + str(rand.randint(1, NUMBER_OF_NODES))
        node2 = RENAME_2 + str(rand.randint(1, NUMBER_OF_NODES))
        G_union.add_edge(node1, node2)
        
    return G_union, EDGES_BETWEEN_PARTITIONS 

In [38]:
# labeling

def label_given_graph(graph, edges):
    
    partition = kernighan_lin_bisection(graph, max_iter = 200)
    
    G_partition1 = graph.subgraph(partition[0])
    G_partition2 = graph.subgraph(partition[1])
    
    # Check Vertex Constraint
    total_vertices = graph.number_of_nodes()
    partition_1_vertices = G_partition1.number_of_nodes()
    partition_2_vertices = G_partition2.number_of_nodes()
    
    min_vertex_bound = total_vertices/2 - total_vertices*0.01
    max_vertex_bound = total_vertices/2 + total_vertices*0.01
    
    if not ((min_vertex_bound <= partition_1_vertices <= max_vertex_bound) and (min_vertex_bound <= partition_2_vertices <= max_vertex_bound)):
        return False
    
    # Check Edge Constraint
    total_edges = graph.number_of_edges()
    partition_1_edges = G_partition1.number_of_edges()
    partition_2_edges = G_partition2.number_of_edges()

    min_cut_edge_amount = total_edges-partition_1_edges-partition_2_edges
    
    if edges >= min_cut_edge_amount:
        return True
    else:
        return False

In [39]:
# Get degree feature
def get_degree_feature(graph):
    return dict(graph.degree())

# Get betweenness centrality feature
def get_betweenness_centrality_feature(graph):
    return nx.betweenness_centrality(graph)

In [40]:
# Generate node features
def generate_node_features(graph, feature_functions):
    feature_dict = {}
    
    # Calculate each feature using the provided functions
    for feature_name, feature_function in feature_functions.items():
        feature_dict[feature_name] = feature_function(graph)
    
    # Create a DataFrame with node features
    node_features = pd.DataFrame(feature_dict, index=graph.nodes())
    
    return node_features

In [41]:
def writeToExcel(mapping_result):
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    file_path = os.path.join(desktop_path, "graph_mapping.xlsx")

    # Check if the file exists
    if os.path.exists(file_path):
        with pd.ExcelWriter(file_path, engine="openpyxl", mode="a") as writer:
            df = pd.DataFrame.from_dict(mapping_result, orient="index")
            startcol = len(writer.sheets["Sheet1"].columns) if "Sheet1" in writer.sheets else 0
            df.to_excel(writer, sheet_name="Sheet1", startcol=startcol, index=True)
    else:
        with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
            df = pd.DataFrame.from_dict(mapping_result, orient="index")
            df.to_excel(writer, sheet_name="Sheet1", index=True)

In [42]:
import differentGraphs as dg

TOTAL_NUMBER_OF_GRAPH_FOR_EACH = 4
NODES_LOW_LIMIT = 60
NODES_HIGH_LIMIT = 200

excel_file_path = 'output.xlsx'

def writeToExcel(embeddings):
    df = pd.DataFrame(embeddings, columns=[f'Column{i}' for i in range(1, len(embeddings[0])+1)])
    if os.path.exists(excel_file_path):
        # İkinci DataFrame'i dosyaya ekleyerek yaz
        with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a') as writer:
            df.to_excel(writer, index=False, header=False)
    else:
        df.to_excel(excel_file_path, index=False)


def get_embedding_SAGE(G):
    
    feature_functions = {
        'degree': get_degree_feature,
        'betweenness_centrality': get_betweenness_centrality_feature,
        # Add more features as needed
    }
    
    # Assuming you have a function to generate node features, modify as needed
    node_features = generate_node_features(G, feature_functions)
    
    # Convert NetworkX graph to StellarGraph with node features
    G_stellar = StellarGraph.from_networkx(G, node_features=node_features)

    # generator
    # batch_size -> number of nodes per batch
    # num_samples -> number of neighbours per layer
    generator = GraphSAGENodeGenerator(G_stellar, batch_size=50, num_samples=[10, 10])
    
    model = GraphSAGE(layer_sizes=[50, 50], generator=generator, aggregator=MeanPoolingAggregator, bias=True, dropout=0.5)
    
    # get input and output tensors
    x_inp, x_out = model.in_out_tensors()

    output_size = 30
    
    # pass the output tensor through the classification layer
    prediction = layers.Dense(units=output_size, activation="linear")(x_out)

    # Combine the GraphSAGE model with the prediction layer
    model = Model(inputs=x_inp, outputs=prediction)

    # Compile the model
    model.compile(optimizer=optimizers.Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=["acc"])

    #model.summary()
    
    # Obtain the graph embedding for all nodes
    node_ids = G_stellar.nodes()
    node_gen = generator.flow(node_ids) # If we have test train vs sets this 3 lines will be copied
    node_embeddings = model.predict(node_gen)
        
    return node_embeddings[1]


def dictionaryToNpArray(embedding):
    array_from_dict = np.array(list(embedding.values()))
    return array_from_dict

def KernighanLinIterationAndSAGEembedding(totalNumberOfIteration, G):
    didItBecomeConnected = False
    graphEmbedding = []
    for j in range(totalNumberOfIteration):
        partition = kernighan_lin_bisection(G,max_iter = 1000)
        G_partition1 = G.subgraph(partition[0])
        G_partition2 = G.subgraph(partition[1])
        if nx.is_connected(G_partition1) and nx.is_connected(G_partition2):
            didItBecomeConnected = True
            total_edges = G.number_of_edges()
            partition_1_edges = G_partition1.number_of_edges()
            partition_2_edges = G_partition2.number_of_edges()
            edgeBetweenSubGraphs = total_edges-partition_1_edges-partition_2_edges
            # Buradaki maksimum edge belirlenecek
            # TODO matemetigi getirilecek
            print(str(edgeBetweenSubGraphs) + "        " + str(j))
            if edgeBetweenSubGraphs < 7: 
                print('girdi1')   
                graphEmbeddingTemp = get_embedding_SAGE(G) 
                graphEmbedding = np.append(graphEmbeddingTemp, 1)
                break
            
        # Sona geldiysek ve hala Yes label alamadıysa No label ver
        # Eger hicbir zaman connected bir sekilde bolunemediyse hicbir sey yapma
        if j == totalNumberOfIteration-1 and didItBecomeConnected:
            print('girdi2')
            graphEmbeddingTemp = get_embedding_SAGE(G) 
            graphEmbedding = np.append(graphEmbeddingTemp, 0)
            break
    return didItBecomeConnected,graphEmbedding    

def dataGenerateAndSave(numberOfNodesLowest, numberOfNodesHighest):
    df=[]
    graphEmbedding = []
    seed = rand.randint(1,1000000)
    numberOfNodes = rand.randint(numberOfNodesLowest,numberOfNodesHighest)

    i = TOTAL_NUMBER_OF_GRAPH_FOR_EACH
    # Number 6 Planar_graph
    while i>0:
        numberOfNodes = rand.randint(numberOfNodesLowest,numberOfNodesHighest)
        numberOfEdges = rand.randint(int(numberOfNodes*1.15),int(numberOfNodes*1.3))
        G = dg.generate_planar_graph(nodes=numberOfNodes,edges=numberOfEdges)
        print(nx.number_of_edges(G))
        print(nx.number_of_nodes(G))
        while not nx.is_connected(G):
            numberOfNodes = rand.randint(numberOfNodesLowest,numberOfNodesHighest)
            numberOfEdges = rand.randint(int(numberOfNodes*1.15),int(numberOfNodes*1.3))
            G = dg.generate_planar_graph(nodes=numberOfNodes,edges=numberOfEdges)

        totalNumberOfIteration = 10
        didItBecomeConnected, graphEmbedding = KernighanLinIterationAndSAGEembedding(totalNumberOfIteration,G)
        if didItBecomeConnected and len(graphEmbedding)>0:
            i = i-1
            if len(df) == 0:
                df = graphEmbedding
            else : 
                df = np.vstack((df, graphEmbedding))

    writeToExcel(df)



def make_graph_connected(G):
    
    while not nx.is_connected(G):
        # Randomly select two nodes from the graph
        node1 = rand.choice(list(G.nodes()))
        node2 = rand.choice(list(G.nodes()))

        # Add an edge between the selected nodes
        G.add_edge(node1, node2)
        
    return G

# Gets a graph and makes is connected
def is_graph_appropriate(graph):
    # Check if the graph is None
    if graph is None:
        return False

    # Check if the graph is empty
    if len(graph.nodes()) == 0:
        return False
    
    return True



dataGenerateAndSave(NODES_LOW_LIMIT,NODES_HIGH_LIMIT)


97
83
5        0
girdi1
225
182
10        0
7        1
9        2
8        3
7        8
8        9
girdi2
198
173
8        1
8        2
7        3
8        4
8        8
8        9
girdi2
97
85
6        6
girdi1


In [43]:
# testing 

TEST_AMOUNT = 1

for i in range(TEST_AMOUNT):
    # Simulate graph creation
    G, edges = simulate_graph_creation(100, 500, 2, 5)

    # Label the graph
    label_val = label_given_graph(G, edges)
    
    
    feature_functions = {
        'degree': get_degree_feature,
        'betweenness_centrality': get_betweenness_centrality_feature,
        # Add more features as needed
    }
    
    # Assuming you have a function to generate node features, modify as needed
    node_features = generate_node_features(G, feature_functions)
    
    # Convert NetworkX graph to StellarGraph with node features
    G_stellar = StellarGraph.from_networkx(G, node_features=node_features)

    # generator
    # batch_size -> number of nodes per batch
    # num_samples -> number of neighbours per layer
    generator = GraphSAGENodeGenerator(G_stellar, batch_size=50, num_samples=[10, 10])
    
    model = GraphSAGE(layer_sizes=[50, 50], generator=generator, aggregator=MeanPoolingAggregator, bias=True, dropout=0.5)
    
    # get input and output tensors
    x_inp, x_out = model.in_out_tensors()

    output_size = 10
    
    # pass the output tensor through the classification layer
    prediction = layers.Dense(units=output_size, activation="linear")(x_out)

    # Combine the GraphSAGE model with the prediction layer
    model = Model(inputs=x_inp, outputs=prediction)

    # Compile the model
    model.compile(optimizer=optimizers.Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=["acc"])

    #model.summary()
    
    # Obtain the graph embedding for all nodes
    node_ids = G_stellar.nodes()
    node_gen = generator.flow(node_ids) # If we have test train vs sets this 3 lines will be copied
    node_embeddings = model.predict(node_gen)
    
    #print(node_embeddings.shape)

    # Map the representation with label_val (assuming label_val is boolean)
    mapping_result = dict(zip(node_ids, node_embeddings[1].flatten()))
    
    # Add the label information to mapping_result
    mapping_result['label'] = label_val
    
    writeToExcel(mapping_result)


KeyError: 0